# ASSISTments Data Mining Competition 2017 - Optional Semester Project

## Imports

In [1]:
import numpy as np
import pandas as pd

In [2]:
DATA_DIR = 'Data/'

## Loading the data

We choose the columns to keep and load all the data into dataframes

In [3]:
not_keep = []

In [4]:
columns_keep = [
    'ITEST_id',
    'startTime',
    'AveKnow',
    'AveCarelessness',
    'AveCorrect',
    'NumActions',
    'AveResBored',
    'AveResEngcon',
    'AveResConf',
    'AveResFrust',
    'AveResOfftask',
    'AveResGaming',
    'actionId',
    'problemId',
    'assignmentId',
    'assistmentId',
    'timeTaken',
    'correct',
    'original',
    'hint',
    'hintCount',
    'hintTotal',
    'scaffold',
    'bottomHint',
    'attemptCount',
    'frIsHelpRequest',
    'frPast5HelpRequest',
    'frPast8HelpRequest',
    'stlHintUsed',
    'past8BottomOut',
    'totalFrPercentPastWrong',
    'totalFrPastWrongCount',
    'frPast5WrongCount',
    'frPast8WrongCount',
    'totalFrTimeOnSkill',
    'timeSinceSkill',
    'frWorkingInSchool',
    'totalFrAttempted',
    'totalFrSkillOpportunities',
    'responseIsFillIn',
    'responseIsChosen',
    'endsWithScaffolding',
    'endsWithAutoScaffolding',
    'frTimeTakenOnScaffolding',
    'frTotalSkillOpportunitiesScaffolding',
    'totalFrSkillOpportunitiesByScaffolding',
    'frIsHelpRequestScaffolding',
    'timeGreater5Secprev2wrong',
    'helpAccessUnder2Sec',
    'timeGreater10SecAndNextActionRight',
    'consecutiveErrorsInRow',
    'totalTimeByPercentCorrectForskill',
    'confidence(BORED)',
    'confidence(CONCENTRATING)',
    'confidence(CONFUSED)',
    'confidence(FRUSTRATED)',
    'confidence(OFF TASK)',
    'confidence(GAMING)',
    'RES_BORED',
    'RES_CONCENTRATING',
    'RES_CONFUSED',
    'RES_FRUSTRATED',
    'RES_OFFTASK',
    'RES_GAMING',
    'Ln-1',
    'Ln'
]

In [5]:
student_logs = pd.concat([
    pd.read_csv(DATA_DIR + 'student_log_' + str(i) + '.csv', usecols=columns_keep) for i in range(1, 11)
], ignore_index=True)

student_logs.head()

/home/dario/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2850: DtypeWarning: Columns (74,75) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


,ITEST_id,AveKnow,AveCarelessness,AveCorrect,NumActions,AveResBored,AveResEngcon,AveResConf,AveResFrust,AveResOfftask,...,confidence(OFF TASK),confidence(GAMING),RES_BORED,RES_CONCENTRATING,RES_CONFUSED,RES_FRUSTRATED,RES_OFFTASK,RES_GAMING,Ln-1,Ln
0,8,0.352416,0.183276,0.483902,1056,0.208389,0.679126,0.115905,0.112408,0.156503,...,0.838710,0.008522,0.376427,0.320317,0.000000,0.0,0.785585,0.000264,0.13,0.0611904
1,8,0.352416,0.183276,0.483902,1056,0.208389,0.679126,0.115905,0.112408,0.156503,...,0.600000,0.047821,0.156027,0.995053,0.887452,0.0,0.468252,0.001483,0.0611904,0.21351
2,8,0.352416,0.183276,0.483902,1056,0.208389,0.679126,0.115905,0.112408,0.156503,...,0.600000,0.047821,0.156027,0.995053,0.887452,0.0,0.468252,0.001483,0.116,0.0333058
3,8,0.352416,0.183276,0.483902,1056,0.208389,0.679126,0.115905,0.112408,0.156503,...,0.204082,0.343996,0.156027,0.744520,0.000000,0.0,0.108417,0.010665,0.116,0.0333058
4,8,0.352416,0.183276,0.483902,1056,0.208389,0.679126,0.115905,0.112408,0.156503,...,0.204082,0.343996,0.156027,0.744520,0.000000,0.0,0.108417,0.010665,0.0333058,0.118386


In [6]:
student_logs.shape

(942816, 66)

In [7]:
train_labels = pd.read_csv('Data/training_label.csv', index_col='ITEST_id').sort_index()
train_labels.drop_duplicates(subset=None, keep='first', inplace=True)

train_labels.head()

,SchoolId,AveCorrect,MCAS,isSTEM
ITEST_id,,,,
9,2,0.438492,32,1
27,1,0.348837,21,0
33,2,0.686391,52,0
35,2,0.379658,34,0
37,3,0.305785,-999,0


In [8]:
test_labels = pd.read_csv(DATA_DIR + 'validation_test_label.csv', index_col='ITEST_id').sort_index()
test_labels.head()

,SchoolId,AveCorrect,MCAS
ITEST_id,,,
9,2,0.438492,32
101,4,0.403553,29
161,1,0.483425,40
164,2,0.256983,9
176,2,0.575949,50


We only keep actions for students for which we have labels in train_labels and test_labels. We also sort by student ID and by startTime of in order to have a chronological suite of actions

In [9]:
student_logs = student_logs.sort_values(by=['ITEST_id', 'startTime'])
del student_logs['startTime']

In [10]:
train_idx = train_labels.index.values
test_idx = test_labels.index.values

student_train_logs = student_logs[student_logs['ITEST_id'].isin(train_idx)]
student_test_logs = student_logs[student_logs['ITEST_id'].isin(test_idx)]
print('Training data shape:', student_train_logs.shape)
print('Test data shape:', student_test_logs.shape)

Training data shape: (251488, 65)
Test data shape: (91038, 65)


In [11]:
student_train_logs.head()

,ITEST_id,AveKnow,AveCarelessness,AveCorrect,NumActions,AveResBored,AveResEngcon,AveResConf,AveResFrust,AveResOfftask,...,confidence(OFF TASK),confidence(GAMING),RES_BORED,RES_CONCENTRATING,RES_CONFUSED,RES_FRUSTRATED,RES_OFFTASK,RES_GAMING,Ln-1,Ln
184889,9,0.185138,0.099734,0.438492,504,0.277149,0.644744,0.098078,0.162771,0.213378,...,0.838710,0.008522,0.376427,0.320317,0.000000,0.000000,0.785585,0.000264,0.212,0.0657639
184890,9,0.185138,0.099734,0.438492,504,0.277149,0.644744,0.098078,0.162771,0.213378,...,0.000000,0.047821,0.156027,0.225154,0.000000,0.009561,0.000000,0.001483,0.0657639,0.211371
184891,9,0.185138,0.099734,0.438492,504,0.277149,0.644744,0.098078,0.162771,0.213378,...,0.280702,0.047821,0.156027,0.665929,0.000000,0.009561,0.149121,0.001483,0.211371,0.482049
184892,9,0.185138,0.099734,0.438492,504,0.277149,0.644744,0.098078,0.162771,0.213378,...,0.600000,0.047821,0.156027,0.780156,0.000000,0.009561,0.468252,0.001483,0.482049,0.759188
184893,9,0.185138,0.099734,0.438492,504,0.277149,0.644744,0.098078,0.162771,0.213378,...,0.578947,0.186970,0.376427,0.195349,0.060808,0.000000,0.440265,0.005797,0.074,0.0449506


Finally we save the data into pickles to use them later

In [12]:
student_test_logs.to_pickle('student_test_logs')
student_train_logs.to_pickle('student_train_logs')